## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-24-21-52-38 +0000,nyt,"Syria Announces Cease-Fire Extension, Hours Af...",https://www.nytimes.com/2026/01/24/world/middl...
1,2026-01-24-21-51-12 +0000,wapo,Shooting victim identified as intensive-care n...,https://www.washingtonpost.com
2,2026-01-24-21-51-12 +0000,wapo,Live updates: Man fatally shot by federal agen...,https://www.washingtonpost.com/nation/2026/01/...
3,2026-01-24-21-42-41 +0000,startribune,Live: State officials decry fatal shooting. ‘W...,https://www.startribune.com/ice-raids-minnesot...
4,2026-01-24-21-41-21 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2026/us/minneapol...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2478/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
87,trump,38
33,ice,20
241,china,13
9,shooting,10
252,canada,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
25,2026-01-24-20-11-38 +0000,nypost,Gavin Newsom launches vicious attack on presid...,https://nypost.com/2026/01/24/us-news/gavin-ne...,94
74,2026-01-24-14-21-45 +0000,cbc,Trump threatens 100% tariffs on all Canadian g...,https://www.cbc.ca/news/politics/trump-tariff-...,87
56,2026-01-24-17-20-53 +0000,nypost,Here’s a look at Arctic drones that can patrol...,https://nypost.com/2026/01/24/us-news/heres-a-...,82
47,2026-01-24-18-30-00 +0000,wsj,Trump Threatens New Tariffs on Canada Over China,https://www.wsj.com/politics/trump-threatens-n...,81
70,2026-01-24-15-14-19 +0000,nypost,Trump threatens Canada with 100% tariff after ...,https://nypost.com/2026/01/24/us-news/trump-th...,78


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
25,94,2026-01-24-20-11-38 +0000,nypost,Gavin Newsom launches vicious attack on presid...,https://nypost.com/2026/01/24/us-news/gavin-ne...
19,64,2026-01-24-20-47-10 +0000,nypost,"Trilateral talks between US, Russia and Ukrain...",https://nypost.com/2026/01/24/world-news/trila...
74,45,2026-01-24-14-21-45 +0000,cbc,Trump threatens 100% tariffs on all Canadian g...,https://www.cbc.ca/news/politics/trump-tariff-...
71,34,2026-01-24-15-01-46 +0000,nypost,Trump reveals to The Post ‘on land’ drug strik...,https://nypost.com/2026/01/24/us-news/trump-re...
26,32,2026-01-24-20-00-42 +0000,nypost,Snow-loving New Yorkers get creative with thei...,https://nypost.com/2026/01/24/us-news/snow-lov...
62,29,2026-01-24-17-06-00 +0000,wsj,China’s Xi Places His Top General Under Invest...,https://www.wsj.com/world/china/chinas-xi-plac...
28,27,2026-01-24-19-58-05 +0000,startribune,University of Minnesota cuts ties with nonprof...,https://www.startribune.com/the-u-signs-agreem...
107,26,2026-01-24-10-50-00 +0000,startribune,NFL championship game picks: NFL’s final four ...,https://www.startribune.com/nfl-championship-g...
2,24,2026-01-24-21-51-12 +0000,wapo,Live updates: Man fatally shot by federal agen...,https://www.washingtonpost.com/nation/2026/01/...
56,24,2026-01-24-17-20-53 +0000,nypost,Here’s a look at Arctic drones that can patrol...,https://nypost.com/2026/01/24/us-news/heres-a-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
